In [ ]:
! pip install datasets transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

## Preparing the dataset

In [ ]:
from datasets import load_dataset
datasets = load_dataset('Sree1994/babylm_100M')

In [ ]:
datasets["train"][10]

## Causal Language modeling

In [ ]:
model_checkpoint = "EleutherAI/gpt-neo-1.3B"

In [ ]:
from transformers import AutoTokenizer
import torch

    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

In [ ]:
tokenized_datasets["train"][1]

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
config = AutoConfig.from_pretrained(
    model_checkpoint,
    vocab_size=len(tokenizer),
    random_init=True
)
model = AutoModelForCausalLM.from_config(config)
model.init_weights()

In [ ]:
#from transformers import AutoModelForCausalLM
#model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Trainer, TrainingArguments

We pass along all of those to the `Trainer` class:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-babylm",
    evaluation_strategy = "epoch",
    learning_rate=4e-5,
    weight_decay=0.001,
    push_to_hub=False,
    num_train_epochs=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["valid"],
)

And we can train our model:

In [ ]:
trainer.train()

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")